In [61]:
import pandas as pd
import glob

# Use glob to get a list of all CSV files in the directory matching the specific pattern
csv_files = glob.glob("Parking_Tags_Data_2022.00[0-7].csv")

# Create an empty list to hold the dataframes
data_frames = []

# Loop through all CSV files and read them into Pandas dataframes
for file in csv_files:
    df = pd.read_csv(file)
    data_frames.append(df)

# Concatenate all dataframes into a single dataframe
merged_df = pd.concat(data_frames, axis=0, ignore_index=True)

In [ ]:
# Group the data by infraction code and sum the number of occurrences of each code
code_counts = merged_df.groupby('infraction_code')['infraction_description'].agg(['size', lambda x: list(set(x))]).reset_index()
code_counts.columns = ['infraction_code', 'ticket_frequency', 'unique_descriptions']

# Sort the code_counts dataframe by ticket frequency in descending order
sorted_counts = code_counts.sort_values('ticket_frequency', ascending=False)

# Get the top 20 infraction types by ticket frequency
top_20 = sorted_counts.head(20)

# Print the top 20 infraction types by ticket frequency
print("Top 20 Infraction Types by Ticket Frequency:")
for i, row in top_20.iterrows():
    print(f"{row['infraction_code']}: {', '.join(row['unique_descriptions'])} ({row['ticket_frequency']})")


In [ ]:
# Group the data by infraction code and sum the revenue generated by each code
code_revenue = merged_df.groupby('infraction_code')['set_fine_amount'].sum().reset_index()
code_revenue.columns = ['infraction_code', 'revenue']

# Join the code_revenue dataframe with the infraction descriptions
merged_revenue = pd.merge(code_revenue, merged_df[['infraction_code', 'infraction_description']].drop_duplicates(), on='infraction_code')

# Sort the merged_revenue dataframe by revenue in descending order
sorted_revenue = merged_revenue.sort_values('revenue', ascending=False)

# Get the top 20 infraction types by revenue
top_20_revenue = sorted_revenue.head(20)

# Print the top 20 infraction types by revenue
print("Top 20 Infraction Types by Revenue:")
for i, row in top_20_revenue.iterrows():
    print(f"{row['infraction_code']}: {row['infraction_description']} (${row['revenue']:.2f})")

# Calculate the total revenue generated by the top 20 infractions
total_revenue = top_20_revenue['revenue'].sum()

# Print the total revenue generated by the top 20 infractions
print(f"\nTotal Revenue Generated by Top 20 Infractions: ${total_revenue:.2f}")


In [ ]:
import matplotlib.pyplot as plt

# Convert date_of_infraction column to a datetime object
merged_df['date_of_infraction'] = pd.to_datetime(merged_df['date_of_infraction'], format='%Y%m%d')

# Extract day of week from datetime object
merged_df['day_of_week'] = merged_df['date_of_infraction'].dt.day_name()

# Group data by day of week and count occurrences
day_counts = merged_df.groupby('day_of_week').size().reset_index(name='counts')

# Set up the plot
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the resulting dataframe as a bar chart
ax.bar(x=day_counts['day_of_week'], height=day_counts['counts'])

# Set the title and axis labels
ax.set_title('Distribution of Infractions by Day of Week')
ax.set_xlabel('Day of Week')
ax.set_ylabel('Number of Infractions')

# Show the plot
plt.show()
